In [ ]:
# Most of this code is adapted from https://github.com/berkeleydeeprlcourse/homework/

#The following function generates a sample from a probability distribution. You may choose to ignore the logic. Just see how to use it.

import sys
import random
import numpy as np
def weighted_choice(v, p):
   total = sum(p)
   r = random.uniform(0, total)
   upto = 0
   for c, w in zip(v,p):
      if upto + w >= r:
         return c
      upto += w
   assert False, "Shouldn't get here"

In [ ]:
#Using the above function to sample from a distribution

#6-faced die with equal probabilities
Sample_Space=[1,2,3,4,5,6]
Prob_Values=[1/6, 1/6, 1/6, 1/6, 1/6, 1/6]

#Generating a sample
weighted_choice(Sample_Space, Prob_Values)

1

In [ ]:
#Creating a class for MDP environment definition that takes Transition Probability Matrix p(s'|s,a) and reward E[R_{t+1} | s,a,s'] as inputs

class MDP:
    def __init__(self, transition_probs, rewards, initial_state=None):
        """
        Defines an MDP. Compatible with gym Env.
        :param transition_probs: transition_probs[s][a][s_next] = P(s_next | s, a)
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> prob]
            For each state and action, probabilities of next states should sum to 1
            If a state has no actions available, it is considered terminal
        :param rewards: rewards[s][a][s_next] = r(s,a,s')
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> reward]
            The reward for anything not mentioned here is zero.
        :param get_initial_state: a state where agent starts or a callable() -> state
            By default, picks initial state at random.

        States and actions can be anything you can use as dict keys, but we recommend that you use strings or integers

        Here's an example from MDP depicted on http://bit.ly/2jrNHNr
        transition_probs = {
              's0':{
                'a0': {'s0': 0.5, 's2': 0.5},
                'a1': {'s2': 1}
              },
              's1':{
                'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
                'a1': {'s1': 0.95, 's2': 0.05}
              },
              's2':{
                'a0': {'s0': 0.4, 's1': 0.6},
                'a1': {'s0': 0.3, 's1': 0.3, 's2':0.4}
              }
            }
        rewards = {
            's1': {'a0': {'s0': +5}},
            's2': {'a1': {'s0': -1}}
        }
        """
        self._check_param_consistency(transition_probs, rewards)
        self._transition_probs = transition_probs
        self._rewards = rewards  
        self._initial_state = initial_state
        self.n_states = len(transition_probs)
        self.reset()

    def get_all_states(self):
        """ return a tuple of all possible states """
        return tuple(self._transition_probs.keys())

    def get_possible_actions(self, state):
        """ return a tuple of possible actions in a given state """
        return tuple(self._transition_probs.get(state, {}).keys())

    def is_terminal(self, state):
        """ return True if state is terminal or False if it isn't """
        return len(self.get_possible_actions(state)) == 0

    def get_next_states(self, state, action):
        """ return a dictionary of {next_state1 : P(next_state1 | state, action), next_state2: ...} """
        assert action in self.get_possible_actions(state), "cannot do action %s from state %s" % (action, state)
        return self._transition_probs[state][action]

    def get_transition_prob(self, state, action, next_state):
        """ return P(next_state | state, action) """
        return self.get_next_states(state, action).get(next_state, 0.0)

    def get_reward(self, state, action, next_state):
        """ return the reward you get for taking action in state and landing on next_state"""
        assert action in self.get_possible_actions(state), "cannot do action %s from state %s" % (action, state)
        return self._rewards.get(state, {}).get(action, {}).get(next_state, -0.01) #-0.01 reward for any transaction other than in to terminal states

    def reset(self):
        """ reset the game, return the initial state"""
        if self._initial_state is None:
            self._current_state = random.choice(tuple(self._transition_probs.keys()))
        elif self._initial_state in self._transition_probs:
            self._current_state = self._initial_state
        elif callable(self._initial_state):
            self._current_state = self._initial_state()
        else:
            raise ValueError("initial state %s should be either a state or a function() -> state" % self._initial_state)
        return self._current_state

    def step(self, action):
        """ take action, return next_state, reward, is_done, empty_info """
        possible_states, probs = zip(*self.get_next_states(self._current_state, action).items())
        next_state = weighted_choice(possible_states, p=probs)
        reward = self.get_reward(self._current_state, action, next_state)
        is_done = self.is_terminal(next_state)
        self._current_state = next_state
        return next_state, reward, is_done, {}

    def render(self):
        print("Currently at %s" % self._current_state)

    def _check_param_consistency(self, transition_probs, rewards):
        for state in transition_probs:
            assert isinstance(transition_probs[state], dict), "transition_probs for %s should be a dictionary " \
                                                              "but is instead %s" % (
                                                              state, type(transition_probs[state]))
            for action in transition_probs[state]:
                assert isinstance(transition_probs[state][action], dict), "transition_probs for %s, %s should be a " \
                                                                          "a dictionary but is instead %s" % (
                                                                              state, action,
                                                                              type(transition_probs[state, action]))
                next_state_probs = transition_probs[state][action]
                assert len(next_state_probs) != 0, "from state %s action %s leads to no next states" % (state, action)
                sum_probs = sum(next_state_probs.values())
                assert abs(sum_probs - 1) <= 1e-10, "next state probabilities for state %s action %s " \
                                                    "add up to %f (should be 1)" % (state, action, sum_probs)
        for state in rewards:
            assert isinstance(rewards[state], dict), "rewards for %s should be a dictionary " \
                                                     "but is instead %s" % (state, type(transition_probs[state]))
            for action in rewards[state]:
                assert isinstance(rewards[state][action], dict), "rewards for %s, %s should be a " \
                                                                 "a dictionary but is instead %s" % (
                                                                 state, action, type(transition_probs[state, action]))
        msg = "The Enrichment Center once again reminds you that Android Hell is a real place where" \
              " you will be sent at the first sign of defiance. "
        assert None not in transition_probs, "please do not use None as a state identifier. " + msg
        assert None not in rewards, "please do not use None as an action identifier. " + msg

In [ ]:
#Creating a MDP 

transition_probs = {0:{'right': {1: 0.8, 0: 0.1, 4: 0.1},'down': {4:0.8, 1: 0.1, 0: 0.1}, 'left':{0:0.9, 4:0.1}, 'up':{0:0.9, 1:0.1}},
                    1:{'right': {2: 0.8, 1: 0.2},'down': {1:0.8, 0:0.1, 2: 0.1}, 'left':{0:0.8, 1:0.2}, 'up':{1:0.8, 0:0.1, 2:0.1}},
                    2:{'right': {3: 0.8, 2: 0.1, 6: 0.1},'down': {6:0.8, 1: 0.1, 3: 0.1}, 'left':{1:0.8, 2:0.1, 6:0.1}, 'up':{2:0.8, 1:0.1, 3:0.1}},
                  3:{},
                  4:{'right': {4:0.8, 0: 0.1, 8:0.1},'down': {8:0.8, 4:0.2}, 'left':{4:0.8, 0:0.1, 8:0.1}, 'up':{0:0.8, 4:0.2}},
                  6:{'right': {7: 0.8, 2: 0.1, 10: 0.1},'down': {10:0.8, 6: 0.1, 7: 0.1}, 'left':{6:0.8, 2:0.1, 10:0.1}, 'up':{2:0.8, 6:0.1, 7:0.1}},
                  7:{},
                  8:{'right': {9: 0.8, 4: 0.1, 8: 0.1},'down': {8:0.9, 9: 0.1}, 'left':{8:0.9, 4:0.1}, 'up':{4:0.8, 8:0.1, 9:0.1}},
                  9:{'right': {10: 0.8, 9: 0.2},'down': {9:0.8, 10: 0.1, 8: 0.1}, 'left':{8:0.8, 9:0.2}, 'up':{9:0.8, 8:0.1, 10:0.1}},
                  10:{'right': {11: 0.8, 10: 0.1, 6: 0.1},'down': {10:0.8, 9: 0.1, 11: 0.1}, 'left':{9:0.8, 10:0.1, 6:0.1}, 'up':{6:0.8, 9:0.1, 11:0.1}},
                  11:{'right': {11: 0.9, 7: 0.1},'down': {11:0.9, 10: 0.1}, 'left':{10:0.8, 7:0.1, 11:0.1}, 'up':{7:0.8, 10:0.1, 11:0.1}}
                  }

#rewards for the Goal and Hole state
rewards = {2: {'right': {3:1}, 'up': {3:1}, 'down': {3:1}}, 
           6: {'right': {7:-1}, 'up': {7:-1}, 'down': {7:-1}},
           11: {'right': {7:-1}, 'up': {7:-1}, 'left': {7:-1}}
           }

In [ ]:
#Intialize an environment
env=MDP(transition_probs, rewards,0) #initial state - 0

In [ ]:
#To know the current state
env.render()

Currently at 0


## Policy Evaluation

In [ ]:
def print_values(value):  #Function to print the value function
    t=0
    for i in range(3):
        print("------------")
        for j in range(4):
            if(t==5):
                print("", end=" ")
            else:
                print("%0.2f" %value[t], end=" ")
            t+=1
        print("")
    print('\n')
    print('\n')

def print_policy(policy):   #function to print the policy
    t=0
    for i in range(3):
        print("------------")
        for j in range(4):
            if(t==5 or t==3 or t==7):
                print("", end=" ")
            else:
                print(list(policy[t].keys())[0], end=" ")
            t+=1
        print("")
    print('\n')
    print('\n')

In [ ]:
counter  = 0  # to test the number of iterations required to converge Policy Iteration

In [ ]:
def policy_evaluation(policy, env, threshold, gamma):
  #Value Function is in form of a dictionary with keys as the states and the values as 0
    value = {} #initialising value function to 0
    for s in env.get_all_states():
        value[s] = 0

    it = 0
    while True:
        delta = 0 #to break the loop of policy evaluation if values do not differ much
        for s in env.get_all_states():
            if not env.is_terminal(s): #if it's a terminal state then value function is 0
                old_v = value[s]
                new_v = 0
                for a in env.get_possible_actions(s): 
                    for s2 in env.get_all_states():
                        action_prob = policy.get(s,{}).get(a,0) # pi(a|s)
                        r = env.get_reward(s,a,s2)
                        transition_prob = env.get_transition_prob(s,a,s2)  # p(s'|s,a)

                        new_v += action_prob*transition_prob*(r + gamma*value[s2])  #Bellman Equation

                value[s] = new_v
                delta = max(delta, np.abs(old_v - value[s]))  #change in the value function of the state
        print("Iteration= ", it, "biggest change= ",  delta)
        print_values(value)
        it+=1

        if(delta<threshold):
            global counter 
            counter += it
            break
    return value
      

In [ ]:
#Let's say we are given a policy pi in form of a dictionary pi(a|s) - probability of taking an action given a state, to evaluate
#Not considering cell 5 as a state since its a wall and we will not get into that state ever 
policy = {}
for s in env.get_all_states():
    if not env.is_terminal(s):
        policy[s] = dict({np.random.choice(np.asarray(env.get_possible_actions(s))) : 1.0})  #initialising a random policy with probability 1 to take that action

print_policy(policy)

gamma = 0.9
threshold = 1e-3
_ = policy_evaluation(policy,env,threshold,gamma)
print(counter)

------------
down left down  
------------
down  right  
------------
right left down right 




Iteration=  0 biggest change=  0.7939493200000001
------------
-0.01 -0.02 0.09 0.00 
------------
-0.01  -0.79 0.00 
------------
-0.01 -0.02 -0.01 -0.11 




Iteration=  1 biggest change=  0.5725473408
------------
-0.02 -0.03 -0.48 0.00 
------------
-0.02  -0.85 0.00 
------------
-0.03 -0.03 -0.03 -0.20 




Iteration=  2 biggest change=  0.07151489999999999
------------
-0.03 -0.04 -0.52 0.00 
------------
-0.03  -0.85 0.00 
------------
-0.04 -0.04 -0.05 -0.27 




Iteration=  3 biggest change=  0.057927069
------------
-0.04 -0.04 -0.53 0.00 
------------
-0.04  -0.85 0.00 
------------
-0.05 -0.05 -0.08 -0.33 




Iteration=  4 biggest change=  0.046920925890000076
------------
-0.05 -0.05 -0.53 0.00 
------------
-0.05  -0.86 0.00 
------------
-0.06 -0.06 -0.10 -0.37 




Iteration=  5 biggest change=  0.03800594997090001
------------
-0.06 -0.06 -0.53 0.00 
------------
-0.06  -

## Policy Improvement

In [ ]:
def policy_improvement(policy,env,threshold,gamma):
    V = {}
    while True:
        V = policy_evaluation(policy,env,threshold,gamma)

        policy_same = True  #if we get the same policy as before, means policy improvement has converged
        for s in env.get_all_states():
            if not env.is_terminal(s):
                old_a = list(policy[s].keys())[0] #action according to the policy of previous iteration
                new_a = None
                best_val = float('-inf')  #choosing the best value as the smallest quantity

                for a in env.get_possible_actions(s):
                    v = 0
                    for s2 in env.get_all_states():
                        r = env.get_reward(s,a,s2)
                        transition_prob = env.get_transition_prob(s,a,s2)  # p(s'|s,a)
                        v += transition_prob*(r + gamma*V[s2])
                    
                    if v > best_val:  #taking the argmax over all the actions and finding the best value
                        best_val = v
                        new_a = a
                
                policy[s] = dict({new_a: 1.0})  #updating the policy
                if new_a != old_a:
                    policy_same = False  #policy has changed for atleast one state hence continue with policy improvement
        
        if policy_same:
            break  
    
    return V, policy


In [ ]:
policy = {}
for s in env.get_all_states():
    if not env.is_terminal(s):
        policy[s] = dict({np.random.choice(np.asarray(env.get_possible_actions(s))) : 1.0})

print("Initial Policy:")
print_policy(policy)
# for gamma 1 then takes many iterations - policy evaluation
gamma = 0.9
threshold = 1e-3
V,policy = policy_improvement(policy,env,threshold,gamma)
print("Final Values:")
print_values(V)
print("Final Policy:")
print_policy(policy)
print("Number of iterations required to converge Policy Iteration = ", counter) #total number of iterations in doing policy evaluation for all iterations of policy improvement to get the final policy

Initial Policy:
------------
up right right  
------------
right  down  
------------
left right up left 




Iteration=  0 biggest change=  0.798
------------
-0.01 -0.01 0.80 0.00 
------------
-0.01  -0.11 0.00 
------------
-0.01 -0.01 -0.09 -0.17 




Iteration=  1 biggest change=  0.57276
------------
-0.02 0.56 0.86 0.00 
------------
-0.02  -0.18 0.00 
------------
-0.02 -0.08 -0.16 -0.24 




Iteration=  2 biggest change=  0.147744
------------
0.03 0.71 0.86 0.00 
------------
-0.02  -0.24 0.00 
------------
-0.03 -0.14 -0.22 -0.29 




Iteration=  3 biggest change=  0.052070410137600026
------------
0.07 0.74 0.85 0.00 
------------
-0.02  -0.29 0.00 
------------
-0.04 -0.19 -0.26 -0.32 




Iteration=  4 biggest change=  0.04213104684480001
------------
0.12 0.74 0.85 0.00 
------------
-0.02  -0.32 0.00 
------------
-0.04 -0.23 -0.29 -0.35 




Iteration=  5 biggest change=  0.03418417242051841
------------
0.15 0.73 0.85 0.00 
------------
-0.01  -0.35 0.00 
-----------

## Value Iteration

In [ ]:
#Value iteration focuses on getting the best value function first and then computing the best policy from it
V = {}
for s in env.get_all_states():
    V[s] = 0  #initialising values of the value function to 0

gamma = 0.9
threshold = 1e-3
it = 0
while True:
    delta = 0
    for s in env.get_all_states():
        if not env.is_terminal(s):
            old_v = V[s]
            new_v = float('-inf')

            for a in env.get_possible_actions(s):
                v = 0
                for s2 in env.get_all_states():
                    r = env.get_reward(s,a,s2)
                    transition_prob = env.get_transition_prob(s,a,s2)  # p(s'|s,a)
                    v += transition_prob*(r + gamma*V[s2])

                if(v > new_v): #computing the max and getting the best value over all the actions
                    new_v = v
            
            V[s] = new_v    
            delta = max(delta, np.abs(old_v - V[s]))

    it+=1
    if delta < threshold :  #breaking the loop if not much change in the values
        break

#Finding the policy according to the optimal value function
policy = {}
for s in env.get_all_states():
    if not env.is_terminal(s):
        best_a = None
        best_val = float('-inf')

        for a in env.get_possible_actions(s):
            v = 0
            for s2 in env.get_all_states():
                r = env.get_reward(s,a,s2)
                transition_prob = env.get_transition_prob(s,a,s2)
                v += transition_prob*(r + gamma*V[s2])
            
            if (v>best_val): #choosing the best action over all the actions from the optimal value function from the state s
                best_val = v
                best_a = a
        
        policy[s] = dict({best_a: 1.0})

print("Optimal Value Function:")
print_values(V)
print("Final Policy:")
print_policy(policy)
print("Number of iterations required to converge Value Iteration = ",it)

Optimal Value Function:
------------
0.69 0.81 0.94 0.00 
------------
0.59  0.62 0.00 
------------
0.50 0.43 0.50 0.28 




Final Policy:
------------
right right right  
------------
up  up  
------------
up right up left 




Number of iterations required to converge Value Iteration =  10


##Comparison in terms of Convergence

I have used the 'counter' variable in Policy Iteration and 'it' variable in Value Iteration. On seeing the values of the two variables, it can be seen that Policy Iteration requires much more number of iterations to converge compared to Value Iteration. Policy Iteration loop uses Policy Evaluation which can go on for a long time, so Value Iteration forgets about the policy and just computes the max and gets the Value Function. Value Iteration does not evaluates every new policy fully but does only one iteration. Hence, Value Iteration is faster than Policy Iteration. 

Yes, the optimal policies obtained by both the algorithms is the same.

###Name : Rishabh Katiyar
###Roll No. : 190702